In [1]:
!pip list

Package                           Version
--------------------------------- --------------
absl-py                           2.1.0
aiohappyeyeballs                  2.6.1
aiohttp                           3.12.13
aiosignal                         1.3.2
altair                            5.5.0
annotated-types                   0.7.0
anyio                             4.4.0
argon2-cffi                       23.1.0
argon2-cffi-bindings              21.2.0
arrow                             1.3.0
asttokens                         2.4.1
astunparse                        1.6.3
async-lru                         2.0.4
attrs                             23.2.0
audioread                         3.0.1
Babel                             2.15.0
beautifulsoup4                    4.12.3
bleach                            6.1.0
blinker                           1.9.0
cachetools                        5.5.2
certifi                           2024.7.4
cffi                              1.16.0
charset-normalizer

In [2]:
!which python


/c/Users/Raunak/anaconda3/envs/ai_tutor/python


In [10]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_multi_view_data(csv_path, base_path='data', img_size=(224, 224)):
    df = pd.read_csv(csv_path)
    X = []
    y = []
    # display(df.head())
    for _, row in df.iterrows():
        child_id = row['child_id']
        gender = row['gender']
        folder_path = os.path.join(base_path, gender, child_id)
        # print(folder_path)
        views = ['_01.jpg', '_02.jpg', '_03.jpg', '_04.jpg']
        images = []

        for view in views:
            view=child_id+view
            img_path = os.path.join(folder_path, view)
            print(img_path)
            if not os.path.exists(img_path):
                continue  # or pad with blank if missing
            img = load_img(img_path, target_size=img_size)
            img = img_to_array(img) / 255.0
            images.append(img)

        if len(images) == 4:
            # Stack along channel dimension: shape will be (H, W, 12)
            stacked_img = tf.concat(images, axis=-1)
            X.append(stacked_img)
            y.append([row['height in cm'], row['weight in grams']])

    return tf.convert_to_tensor(X), tf.convert_to_tensor(y, dtype=tf.float32)


In [15]:
from tensorflow.keras import layers, models

def build_multi_view_cnn(input_shape=(224, 224, 12)):  # 4 views × 3 channels
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),

        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),

        layers.Conv2D(128, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2)  # Output: [height, weight]
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [12]:
X, y = load_multi_view_data("labels.csv")
X

data\f\child0001\child0001_01.jpg
data\f\child0001\child0001_02.jpg
data\f\child0001\child0001_03.jpg
data\f\child0001\child0001_04.jpg
data\f\child0002\child0002_01.jpg
data\f\child0002\child0002_02.jpg
data\f\child0002\child0002_03.jpg
data\f\child0002\child0002_04.jpg
data\m\child0003\child0003_01.jpg
data\m\child0003\child0003_02.jpg
data\m\child0003\child0003_03.jpg
data\m\child0003\child0003_04.jpg
data\m\child0004\child0004_01.jpg
data\m\child0004\child0004_02.jpg
data\m\child0004\child0004_03.jpg
data\m\child0004\child0004_04.jpg
data\m\child0005\child0005_01.jpg
data\m\child0005\child0005_02.jpg
data\m\child0005\child0005_03.jpg
data\m\child0005\child0005_04.jpg
data\m\child0006\child0006_01.jpg
data\m\child0006\child0006_02.jpg
data\m\child0006\child0006_03.jpg
data\m\child0006\child0006_04.jpg
data\m\child0007\child0007_01.jpg
data\m\child0007\child0007_02.jpg
data\m\child0007\child0007_03.jpg
data\m\child0007\child0007_04.jpg
data\f\child0008\child0008_01.jpg
data\f\child00

<tf.Tensor: shape=(91, 224, 224, 12), dtype=float32, numpy=
array([[[[0.6509804 , 0.6666667 , 0.67058825, ..., 0.6901961 ,
          0.7058824 , 0.70980394],
         [0.654902  , 0.67058825, 0.6745098 , ..., 0.6901961 ,
          0.7058824 , 0.70980394],
         [0.6627451 , 0.6666667 , 0.6745098 , ..., 0.69803923,
          0.7019608 , 0.70980394],
         ...,
         [0.70980394, 0.7176471 , 0.7137255 , ..., 0.7490196 ,
          0.7490196 , 0.7411765 ],
         [0.7019608 , 0.7019608 , 0.70980394, ..., 0.7294118 ,
          0.7294118 , 0.7294118 ],
         [0.7058824 , 0.7058824 , 0.7137255 , ..., 0.7411765 ,
          0.7411765 , 0.7411765 ]],

        [[0.6666667 , 0.67058825, 0.6784314 , ..., 0.69803923,
          0.7019608 , 0.70980394],
         [0.6627451 , 0.6666667 , 0.6745098 , ..., 0.7058824 ,
          0.70980394, 0.7176471 ],
         [0.6627451 , 0.67058825, 0.6666667 , ..., 0.70980394,
          0.7176471 , 0.7137255 ],
         ...,
         [0.69411767, 0.7019

In [19]:
model = build_multi_view_cnn()
model.fit(X, y, epochs=50, validation_split=0.2, batch_size=16)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 182245008.0000 - mae: 9313.9238 - val_loss: 148000384.0000 - val_mae: 8519.2861
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 147465648.0000 - mae: 8384.4238 - val_loss: 11931336.0000 - val_mae: 2579.9968
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 30283836.0000 - mae: 3739.8525 - val_loss: 7413337.5000 - val_mae: 2012.7699
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 16998298.0000 - mae: 2789.6919 - val_loss: 19078480.0000 - val_mae: 2912.2034
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 24049178.0000 - mae: 3114.4810 - val_loss: 10515011.0000 - val_mae: 2136.2466
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 966ms/step - loss: 16945772.0000 - mae: 2640.5132 - val_loss: 7065816.5000 - val_mae: 1788.4613
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 903ms/step - loss: 13161754.0000 - mae: 2343.4753 - val_loss: 7249068.0000 - val_mae: 1760.7831
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 981ms/step - loss: 12030

In [18]:
y

<tf.Tensor: shape=(91, 2), dtype=float32, numpy=
array([[  100. , 15000. ],
       [  118. , 28400. ],
       [   95.7, 19000. ],
       [  109.5, 21400. ],
       [   93.5, 12100. ],
       [   94. , 13500. ],
       [  100. , 14200. ],
       [  116. , 22000. ],
       [  102. , 14400. ],
       [  104. , 17400. ],
       [   86. , 12300. ],
       [  116.5, 21700. ],
       [   99. , 15400. ],
       [  114. , 17300. ],
       [  107. , 16000. ],
       [  121. , 23800. ],
       [  124. , 24100. ],
       [  114. , 21900. ],
       [  122. , 22200. ],
       [  102. , 15200. ],
       [  123. , 24000. ],
       [  116. , 18400. ],
       [   90. , 12600. ],
       [  113. , 17900. ],
       [   92. , 12000. ],
       [  101. , 15500. ],
       [  127. , 30700. ],
       [  122. , 21800. ],
       [  109. , 18800. ],
       [   90. , 13000. ],
       [  100. , 14500. ],
       [  135. , 26200. ],
       [  116. , 20000. ],
       [  116. , 19000. ],
       [   95. , 13000. ],
      

In [20]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# After training
y_pred = model.predict(X)
y_true = y.numpy() if isinstance(y, tf.Tensor) else y

# Compute R² score
r2 = r2_score(y_true, y_pred)

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 552ms/step
R² Score: -0.7751
RMSE: 3026.4951


In [21]:
y_true, y_pred

(array([[  100. , 15000. ],
        [  118. , 28400. ],
        [   95.7, 19000. ],
        [  109.5, 21400. ],
        [   93.5, 12100. ],
        [   94. , 13500. ],
        [  100. , 14200. ],
        [  116. , 22000. ],
        [  102. , 14400. ],
        [  104. , 17400. ],
        [   86. , 12300. ],
        [  116.5, 21700. ],
        [   99. , 15400. ],
        [  114. , 17300. ],
        [  107. , 16000. ],
        [  121. , 23800. ],
        [  124. , 24100. ],
        [  114. , 21900. ],
        [  122. , 22200. ],
        [  102. , 15200. ],
        [  123. , 24000. ],
        [  116. , 18400. ],
        [   90. , 12600. ],
        [  113. , 17900. ],
        [   92. , 12000. ],
        [  101. , 15500. ],
        [  127. , 30700. ],
        [  122. , 21800. ],
        [  109. , 18800. ],
        [   90. , 13000. ],
        [  100. , 14500. ],
        [  135. , 26200. ],
        [  116. , 20000. ],
        [  116. , 19000. ],
        [   95. , 13000. ],
        [  118. , 18

In [22]:
#only for height prediction as weight seems off
from tensorflow.keras import layers, Sequential

def build_height_only_cnn(input_shape=(224, 224, 12)):
    model = Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),

        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),

        layers.Conv2D(128, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1)  # Output: height only
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [27]:
y_height_only = y[:, 0:1]  # Keeps shape (num_samples, 1)
y_height_only

<tf.Tensor: shape=(91, 1), dtype=float32, numpy=
array([[100. ],
       [118. ],
       [ 95.7],
       [109.5],
       [ 93.5],
       [ 94. ],
       [100. ],
       [116. ],
       [102. ],
       [104. ],
       [ 86. ],
       [116.5],
       [ 99. ],
       [114. ],
       [107. ],
       [121. ],
       [124. ],
       [114. ],
       [122. ],
       [102. ],
       [123. ],
       [116. ],
       [ 90. ],
       [113. ],
       [ 92. ],
       [101. ],
       [127. ],
       [122. ],
       [109. ],
       [ 90. ],
       [100. ],
       [135. ],
       [116. ],
       [116. ],
       [ 95. ],
       [118. ],
       [113. ],
       [ 86. ],
       [121. ],
       [125. ],
       [126. ],
       [101. ],
       [121. ],
       [ 92. ],
       [115. ],
       [ 94.3],
       [107. ],
       [102. ],
       [117. ],
       [116. ],
       [129. ],
       [120. ],
       [109. ],
       [125. ],
       [125.5],
       [120. ],
       [109. ],
       [119. ],
       [ 99.5],
       

In [28]:
# Build the height-only CNN model
model = build_height_only_cnn(input_shape=(224, 224, 12))  # 12 channels from 4 views

# Train the model
history = model.fit(X, y_height_only, epochs=30, batch_size=16, validation_split=0.2)


Epoch 1/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 883ms/step - loss: 7341.2217 - mae: 76.3490 - val_loss: 1069.1637 - val_mae: 31.6151
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 922ms/step - loss: 1084.5128 - mae: 30.3288 - val_loss: 424.4706 - val_mae: 18.2741
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 885ms/step - loss: 515.3454 - mae: 18.3896 - val_loss: 313.5777 - val_mae: 15.6906
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 896ms/step - loss: 281.3746 - mae: 13.0129 - val_loss: 84.1479 - val_mae: 7.8983
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 220.6671 - mae: 11.5826 - val_loss: 97.5158 - val_mae: 8.0103
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 269.7106 - mae: 12.8273 - val_loss: 219.4051 - val_mae: 12.4996
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 997ms/step - loss: 249.7331 - mae: 13.7563 - val_loss: 84.7841 - val_mae: 7.6420
Epoch 8/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 972ms/step - loss: 158.8829 - mae: 10.0855 - val_loss: 189.9106 - val_mae: 11.2423
Epoch 9/30
5/5 ━━━━━━━━━━━━━━━━━━

In [29]:
# Predict on entire dataset (or test split if available)
y_pred = model.predict(X).flatten()
y_true = y_height_only.numpy().flatten()  # Convert tensor to flat array

# Mean Absolute Error (already shown during training)
mae = np.mean(np.abs(y_true - y_pred))

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# R² Score
r2 = r2_score(y_true, y_pred)

print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 307ms/step
MAE:  8.1565
RMSE: 11.2166
R²:   0.1790
